# Save Data to Parquet
The purpose of this notebook is to read data from multiple .csv files and save it to a Parquet file format. Additionally, the data is being processed by filling  wrong characters with NA (null) values and casting data types to optimize storage and enhance reading efficiency.

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import polars as pl
# import pyarrow
# import pyarrow.dataset
from seaborn import set_style
set_style("darkgrid")

In [2]:
DATA_PATH = "data/precipitation/"


COL_NAMES = [
    "Kod stacji",
    "Nazwa Stacji",
    "Rok",
    "Miesiac",
    "Dzien",
    "Suma Opadow [mm]",
    "Status pomiaru SMDB",
    "Rodzaj opadu [S/W/ ]",
    "Wysokosc pokrywy snieznej cm",
    "Status pomiaru PKSN",
    "Wysokosc swiezospadlego sniedu [cm]",
    "Status pomiaru HSS",
    "Gatunek śniegu  [kod]",
    "Status pomiaru GATS",
    "Rodzaj pokrywy śnieżnej [kod]",
    "Status pomiaru RPSN"
    ]

COL_DTYPE_MAP = {
    "Kod stacji":pl.String,
    "Nazwa Stacji":pl.String,
    "Rok":pl.Int64,
    "Miesiac":pl.Int64,
    "Dzien":pl.Int64,
    "Suma Opadow [mm]":pl.Float64,
    "Status pomiaru SMDB":pl.String,
    "Rodzaj opadu [S/W/ ]":pl.String,
    "Wysokosc pokrywy snieznej cm":pl.Float64,
    "Status pomiaru PKSN":pl.String,
    "Wysokosc swiezospadlego sniedu [cm]":pl.Float64,
    "Status pomiaru HSS":pl.String,
    "Gatunek śniegu  [kod]":pl.String,
    "Status pomiaru GATS":pl.String,
    "Rodzaj pokrywy śnieżnej [kod]":pl.String,
    "Status pomiaru RPSN":pl.String
}


In [3]:
def read_csv_files_in_directory(directory,col_names):
    # Initialize an empty list to store DataFrames
    all_dataframes = []
    counter = 0
    # Walk through the directory tree
    for root, dirs, files in os.walk(directory):
        for file in files:
            # Check if the file is a CSV file
            if file.endswith('.csv'):
                # Construct the file path
                file_path = os.path.join(root, file)
                
                # Read the CSV file with error handling and data type specification
                try:
                    df = pl.read_csv(file_path, ignore_errors=True, new_columns=col_names)
                except Exception as e:
                    print(f"Error reading {file}: {e}")
                    continue
                counter += 1
                print(counter)
                print(df.shape)
                # Append the DataFrame to the list
                all_dataframes.append(df)
    
    return all_dataframes

def fill_string_with_na(df, columns):
    df = df.clone()
    for col in columns:
        series_slc = df[col]
        if isinstance(series_slc.dtype, pl.String):
            series_slc = series_slc.replace("", None).replace(".",None).replace("*",None)
            df = df.with_columns([col], series_slc)  # Use with_column to update the DataFrame
    return df

def map_dtypes(df,map):
    for col in  map.keys():
        df = df.with_columns([col], df[col].cast(map[col]))

    return df



In [4]:
all_data = read_csv_files_in_directory(DATA_PATH,col_names=COL_NAMES)

1
(5499, 16)
2
(76031, 16)
3
(110465, 16)
4
(84408, 16)
5
(108508, 16)
6
(121401, 16)
7
(135151, 16)
8
(129962, 16)
9
(158169, 16)
10
(117004, 16)
11
(157503, 16)
12
(158479, 16)
13
(178611, 16)
14
(156370, 16)
15
(162950, 16)
16
(191890, 16)
17
(189389, 16)
18
(187606, 16)
19
(198563, 16)
20
(169556, 16)
21
(223119, 16)
22
(191845, 16)
23
(180508, 16)
24
(207562, 16)
25
(217805, 16)
26
(187866, 16)
27
(202470, 16)
28
(214115, 16)
29
(225436, 16)
30
(207133, 16)
31
(228772, 16)
32
(222965, 16)
33
(154756, 16)
34
(196623, 16)
35
(195306, 16)
36
(221304, 16)
37
(184000, 16)
38
(207705, 16)
39
(217351, 16)
40
(184105, 16)
41
(203362, 16)
42
(193885, 16)
43
(180476, 16)
44
(189205, 16)
45
(191295, 16)
46
(175778, 16)
47
(172339, 16)
48
(171735, 16)
49
(183342, 16)
50
(170910, 16)
51
(165331, 16)
52
(13379, 16)
53
(14160, 16)
54
(15935, 16)
55
(16199, 16)
56
(10032, 16)
57
(16454, 16)
58
(16809, 16)
59
(11757, 16)
60
(19319, 16)
61
(10824, 16)
62
(17910, 16)
63
(20510, 16)
64
(15488, 16)
65

In [5]:
len(all_data)

317

In [6]:
for i in range(len(all_data)):
    all_data[i] = fill_string_with_na(all_data[i],COL_NAMES)
    all_data[i] = map_dtypes(all_data[i],COL_DTYPE_MAP)
    all_data[i] = all_data[i][COL_NAMES]

## CONCATENATED

In [7]:
final_df = pl.concat(all_data)

In [8]:
final_df

Kod stacji,Nazwa Stacji,Rok,Miesiac,Dzien,Suma Opadow [mm],Status pomiaru SMDB,Rodzaj opadu [S/W/ ],Wysokosc pokrywy snieznej cm,Status pomiaru PKSN,Wysokosc swiezospadlego sniedu [cm],Status pomiaru HSS,Gatunek śniegu [kod],Status pomiaru GATS,Rodzaj pokrywy śnieżnej [kod],Status pomiaru RPSN
str,str,i64,i64,i64,f64,str,str,f64,str,f64,str,str,str,str,str
"""249180120""",null,1950,11,5,2.1,null,null,0.0,"""8""",0.0,"""8""",null,"""8""",null,"""8"""
"""249180120""",null,1950,11,7,2.9,null,null,0.0,"""8""",0.0,"""8""",null,"""8""",null,"""8"""
"""249180120""",null,1950,11,8,0.5,null,null,0.0,"""8""",0.0,"""8""",null,"""8""",null,"""8"""
"""249180120""",null,1950,11,11,0.6,null,null,0.0,"""8""",0.0,"""8""",null,"""8""",null,"""8"""
"""249180120""",null,1950,11,14,12.6,null,null,0.0,"""8""",0.0,"""8""",null,"""8""",null,"""8"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""353150210""",null,2023,2,23,0.7,null,"""W""",0.0,"""9""",0.0,"""9""",null,"""9""",null,null
"""353150210""",null,2023,2,24,4.1,null,"""W""",0.0,"""9""",0.0,"""9""",null,"""9""",null,null
"""353150210""",null,2023,2,25,7.0,null,"""S""",0.0,"""9""",0.0,"""9""",null,"""9""",null,null


In [9]:
final_df = final_df.with_columns(
    pl.date(
     pl.col("Rok").cast(pl.Int64),
     pl.col("Miesiac").cast(pl.Int64),
     pl.col("Dzien").cast(pl.Int64)   
    ).alias("data_opadu")
) 

In [10]:
final_df["Kod stacji"].value_counts()

Kod stacji,count
str,u32
"""250150080""",13437
"""251220160""",11424
"""253230080""",336
"""253190210""",10378
"""254160040""",12487
…,…
"""249190810""",1207
"""251169998""",7106
"""250160150""",14635


In [11]:
final_df["Kod stacji"].is_null().sum()

0

In [12]:
final_df

Kod stacji,Nazwa Stacji,Rok,Miesiac,Dzien,Suma Opadow [mm],Status pomiaru SMDB,Rodzaj opadu [S/W/ ],Wysokosc pokrywy snieznej cm,Status pomiaru PKSN,Wysokosc swiezospadlego sniedu [cm],Status pomiaru HSS,Gatunek śniegu [kod],Status pomiaru GATS,Rodzaj pokrywy śnieżnej [kod],Status pomiaru RPSN,data_opadu
str,str,i64,i64,i64,f64,str,str,f64,str,f64,str,str,str,str,str,date
"""249180120""",null,1950,11,5,2.1,null,null,0.0,"""8""",0.0,"""8""",null,"""8""",null,"""8""",1950-11-05
"""249180120""",null,1950,11,7,2.9,null,null,0.0,"""8""",0.0,"""8""",null,"""8""",null,"""8""",1950-11-07
"""249180120""",null,1950,11,8,0.5,null,null,0.0,"""8""",0.0,"""8""",null,"""8""",null,"""8""",1950-11-08
"""249180120""",null,1950,11,11,0.6,null,null,0.0,"""8""",0.0,"""8""",null,"""8""",null,"""8""",1950-11-11
"""249180120""",null,1950,11,14,12.6,null,null,0.0,"""8""",0.0,"""8""",null,"""8""",null,"""8""",1950-11-14
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""353150210""",null,2023,2,23,0.7,null,"""W""",0.0,"""9""",0.0,"""9""",null,"""9""",null,null,2023-02-23
"""353150210""",null,2023,2,24,4.1,null,"""W""",0.0,"""9""",0.0,"""9""",null,"""9""",null,null,2023-02-24
"""353150210""",null,2023,2,25,7.0,null,"""S""",0.0,"""9""",0.0,"""9""",null,"""9""",null,null,2023-02-25


In [13]:
final_df.write_parquet("data/concatenated.parquet")